<a href="https://colab.research.google.com/github/YapingWu/GoogleColab/blob/main/adjust_param.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 导入需要的包

In [14]:
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding
from keras.layers import Dropout
from keras import optimizers
from keras.callbacks import TensorBoard
from sklearn.model_selection import train_test_split
import time
import numpy as np

# 加载数据




In [4]:
SEED = 7

In [5]:
# 数据相关参数
data_names = ['myspace', 'phpbb', 'rockyou']
vocab_sizes = [73, 56, 251]
max_lengths = [35, 21, 41]

i = 1
data_name = data_names[i]
vocab_size = vocab_sizes[i]
max_length = max_lengths[i]
print("dataname：%s, vocab_size：%d, max_length：%d" % (data_name, vocab_size, max_length))

dataname：phpbb, vocab_size：56, max_length：21


In [6]:
print("加载数据：")
X = np.loadtxt('/content/' + data_name + 'x.txt')
y = np.loadtxt('/content/' + data_name + 'y.txt')
print("X Shape: %s, y Shape: %s" % (X.shape, y.shape))

print("划分训练集、验证集和测试集：")
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=SEED)
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=0.5, random_state=SEED)
print("X_train Shape: %s, y_train Shape: %s" % (X_train.shape, y_train.shape))
print("X_val Shape: %s, y_val Shape: %s" % (X_val.shape, y_val.shape))
print("X_test Shape: %s, y_test Shape: %s" % (X_test.shape, y_test.shape))

加载数据：
X Shape: (15322, 20), y Shape: (15322, 56)
划分训练集、验证集和测试集：
X_train Shape: (12257, 20), y_train Shape: (12257, 56)
X_val Shape: (1532, 20), y_val Shape: (1532, 56)
X_test Shape: (1533, 20), y_test Shape: (1533, 56)


# 调整网络参数

## 调整模型参数
全连接层个数、lstm层个数和每层神经元个数

### 设置参数

In [7]:
# 设置参数
epochs = 30
batch_size = 128
# 全连接层个数
dense_layers = [1, 2]
# 每层神经元个数
layer_sizes = [32, 64, 128]
# lstm层数
lstm_layers = [1, 2, 3]

log_dir = '/content/logs/' + data_name +'/'

### 建立和训练模型

In [ ]:
# 建立和训练模型
for dense_layer in dense_layers:
  for layer_size in layer_sizes:
    for lstm_layer in lstm_layers:
      
      NAME = log_dir + "{}-lstm-{}-notes-{}-dense".format(lstm_layer, layer_size, dense_layer)
      tensorboard = TensorBoard(log_dir=NAME)
      print(NAME)
      
      model = Sequential()
      model.add(Embedding(input_dim=vocab_size, output_dim=10, input_length=max_length-1))
      
      for l in range(lstm_layer - 1):
        model.add(LSTM(layer_size, return_sequences=True))
      model.add(LSTM(layer_size, return_sequences=False))
      
      for l in range(dense_layer - 1):
        model.add(Dense(units=layer_size, activation='relu'))
      model.add(Dense(vocab_size, activation='softmax'))

      model.summary()
      
      adam = optimizers.Adam(lr = 0.01)
      model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])
      
      model.fit(X_train, y_train, 
                validation_data=(X_val, y_val), 
                epochs=epochs, 
                batch_size=batch_size, 
                verbose=1,
                callbacks=[tensorboard])
      loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
      print("Model Accuracy: %.2f%%, Loss: %.2f" % (accuracy * 100, loss))

### tensorboard中查看

In [ ]:
%tensorboard --logdir /content/logs/phpbb

## 调整batch_size、epoch

### [1, 32, 1]
dense_layer、layer_size和lstm_layer的组合为[1, 32, 1]

In [11]:
# 设置参数
epochs = 150

dense_layer = 1
layer_size = 256
lstm_layer = 1
log_dir = '/content/logs/' + data_name +'/1-256-1/'

batch_sizes = [128, 256]

In [16]:
# 建立和训练模型
for batch_size in batch_sizes:

  NAME = log_dir + str(batch_size) + '-2'
  tensorboard = TensorBoard(log_dir=NAME)
  print(NAME)
  
  model = Sequential()
  model.add(Embedding(input_dim=vocab_size, output_dim=10, input_length=max_length-1))
  model.add(Dropout(0.2))
  
  for l in range(lstm_layer - 1):
    model.add(LSTM(layer_size, return_sequences=True))
    model.add(Dropout(0.2))
  model.add(LSTM(layer_size, return_sequences=False))
  model.add(Dropout(0.2))
  
  for l in range(dense_layer - 1):
    model.add(Dense(units=layer_size, activation='relu'))
    model.add(Dropout(0.2))
  model.add(Dense(vocab_size, activation='softmax'))

  model.summary()
  
  adam = optimizers.Adam(lr = 0.01)
  model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])
  
  model.fit(X_train, y_train, 
            validation_data=(X_val, y_val), 
            epochs=epochs, 
            batch_size=batch_size, 
            verbose=1,
            callbacks=[tensorboard])
  loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
  print("Model Accuracy: %.2f%%, Loss: %.2f" % (accuracy * 100, loss))

/content/logs/phpbb/1-256-1/128-2
Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 20, 10)            560       
_________________________________________________________________
dropout_1 (Dropout)          (None, 20, 10)            0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 256)               273408    
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 56)                14392     
Total params: 288,360
Trainable params: 288,360
Non-trainable params: 0
_________________________________________________________________
Epoch 1/150
96/96 [==============================] - 15s 133ms/step - loss: 2.61

NameError: ignored

### [2, 128, 3]
dense_layer、layer_size和lstm_layer的组合为[2, 128, 3]

In [ ]:
# 设置参数
epochs = 50

dense_layer = 2
layer_size = 128
lstm_layer = 3
log_dir = '/content/logs/' + data_name +'/2-128-3/'

batch_sizes = [32, 64, 128, 256, 512]

In [ ]:
# 建立和训练模型
for batch_size in batch_sizes:
  NAME = log_dir + str(batch_size)
  tensorboard = TensorBoard(log_dir=NAME)
  print(NAME)
  
  model = Sequential()
  model.add(Embedding(input_dim=vocab_size, output_dim=10, input_length=max_length-1))
  
  for l in range(lstm_layer - 1):
    model.add(LSTM(layer_size, return_sequences=True))
  model.add(LSTM(layer_size, return_sequences=False))
  
  for l in range(dense_layer - 1):
    model.add(Dense(units=layer_size, activation='relu'))
  model.add(Dense(vocab_size, activation='softmax'))

  model.summary()
  
  adam = optimizers.Adam(lr = 0.01)
  model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])
  
  model.fit(X_train, y_train, 
            validation_data=(X_val, y_val), 
            epochs=epochs, 
            batch_size=batch_size, 
            verbose=1,
            callbacks=[tensorboard])
  loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
  print("Model Accuracy: %.2f%%, Loss: %.2f" % (accuracy * 100, loss))

# 需要的命令

## 解压文件

In [ ]:
!unzip /content/myspace.zip

Archive:  /content/myspace.zip
  inflating: myspacey.txt            
  inflating: myspacex.txt            


In [1]:
!unzip /content/phpbb.zip

Archive:  /content/phpbb.zip
  inflating: phpbby.txt              
  inflating: phpbbx.txt              


In [ ]:
!unzip /content/rockyou.zip

Archive:  /content/rockyou.zip
  inflating: rockyouy.txt            
  inflating: rockyoux.txt            


## 使用tensorboard

In [2]:
%load_ext tensorboard

In [ ]:
%tensorboard --logdir logs